# Post Download Processing Script
This script is intended to process the data from the US Census Geo-Tiger FTP Site. Please note that this script must be run in ArcPro's Jupyter Lab, and not the Anaconda Jupyter Lab in order to successfully import all modules.

## Copy shapes to geodatabase (merge into national or statewide extents)


In [15]:
import os
import sys
import zipfile
import string
import arcpy

#this is where the downloaded census data is in OS
working_directory = r"C:/Users/and04671/Documents/Data_Files"

#types of census data defined for NHGIS in previous years
folders_old = ['AIANNH','AITSN','ANRC','BG','CBSA','CD','CNECTA','CONCITY','COUNTY',
        'COUSUB','CSA','ELSD','METDIV','NECTA','NECTADIV','PLACE','PUMA',
        'SCSD','SLDL','SLDU','STATE','SUBMCD','TBG','TRACT','TTRACT','UAC',
        'UNSD','ZCTA5',]

#types of census data newly defined for NHGIS 2018          
folders_new = ['AREAWATER','LINEARWATER','COASTLINE','PRIMARYROADS',
        'PRISECROADS','RAILS','ROADS']


#this specifies where the geodatabase will be placed
out_db = r"C:/Users/and04671/Documents/Data_Files/TIGER2018.gdb"

#puts shapefiles into the geodatabase, merging shapefiles into national extents (this is not being done for water, roads)
for folder in folders_old:
    shp_dir = working_directory + "/" + folder
    print(shp_dir)
  # establish the right'sandbox' to work in
    arcpy.env.workspace = shp_dir
    
    #compile a lit of all feature classes as defined by arg
    shps = arcpy.ListFeatureClasses("*")
    
    #if there is one shapefile in the folder
    if len(shps) == 1:
        
        #the name of the shapefile is shp_dir + first shapefile in folder (the 0th)
        shp_file = shp_dir + "/" + shps[0]
        shp_name = shps[0]
        #strip the file type off name
        shp_name = shp_name.strip(".shp")
        #shp_name = shp_name.strip(".")
        
        #name the new out feature class
        out_fc = out_db + "/" + folder
        #copy the shapefile from original to geodatabase
        arcpy.CopyFeatures_management(shp_file, out_fc)
    
    #if there is more than one shapefile in the folder
    else:
        
        #out FC is database + "US extent" + folder (data type)  
        out_fc = out_db + "tl_2018_us_" + folder.lower()
        #merge shapefiles at location and move to geodatabase
        arcpy.Merge_management(shps,out_fc)


C:/Users/and04671/Documents/Data_Files/AIANNH


ExecuteError: ERROR 000210: Cannot create output C:/Users/and04671/Documents/Data_Files/TIGER2018.gdb/AIANNH
Failed to execute (CopyFeatures).


## Add GISjoin

In [ ]:
import os
import sys
import zipfile
import string
import arcpy
#see GIS JOIN pattern for # reasoning
working_directory = r"C:/Users/and04671/Documents/Data_Files"

out_db = r"C:/Users/and04671/Documents/Data_Files/TIGER2018.gdb"

dir_dict = {'AIANHH':5,'AITSN':8,'ANRC':6,'BG':15,'CBSA':6,'CD115':6,'CNECTA':4,'CONCITY':9,'COUNTY':8,
            'COUSUB':13,'CSA':4,'DIVISION':2,'ELSD':9,'METDIV':11,'NATION':2,'NECTA':6,'NECTADIV':11,
            'PLACE':9,'PUMA':9,'SCSD':9,'SLDL':7,'SLDU':7,'STATE':4,'SUBMCD':13,'TBG':12,'TRACT':14,
            'TTRACT':11,'UAC10':6,'UNSD':9,'ZCTA510':6,'TRUST':5,'RES_ONLY':5,'REGION':2}

'''     'AREAWATER','LINEARWATER','COASTLINE','PRIMARYROADS',
        'PRISECROADS','RAILS','ROADS'
'''

#note that the original folder pulls did not have Division, Nation, Trust, Res Only, Region as these could not be found...
#on the census site
#character # should be rihgt. SUBMCD was 13, not 18 as listed

new_field = "GISJOIN"
#establish sandbox to work in
arcpy.env.workspace = out_db
#this looks at the (key,item) pairs in the dict
for key, textfield in dir_dict.items():
    #make the key name lowercase in a new variable
    key_lower = key.lower()
    fc = arcpy.ListFeatureClasses("*") 
    fc = fc + key_lower
    print(fc)
'''
    fc_path = out_db + fc[0]
    arcpy.AddField_management(fc_path,new_field,"TEXT","","",textfield,"","","","")
'''    


## Reprojection

In [27]:
from glob import glob
import os
import arcpy

# Source location - where are the shapefiles coming from? 
# Change this source location to your source location
# Make sure to copy file paths in using \\ or / (and r) as path separators instead of \ (escape key) or use r (raw) to escape 
source = r"C:\Users\and04671\Documents\Data_Files"
for each in glob(os.path.join(source, "*")):
    print(each)
#glob looks for files ending in .shp in the specificed folder location from src (source)
    for file in glob(os.path.join(each, "*.shp")):
        #print(file)
        # input data is in GCS, not projected
        input_features = file
        # output data
        # rename each output - I typically just add "proj" or "projected" to the original name 
        output_feature_class = os.path.join(source,file[:-4]+'_proj.shp')
        print(output_feature_class)
        # set the Coordinate System - don't change 
        out_coordinate_system = arcpy.SpatialReference('USA Contiguous Albers Equal Area Conic')
        # run the tool
        arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)


# create a spatial reference object for the output coordinate system
#out_coordinate_system = arcpy.SpatialReference('USA Contiguous Albers Equal Area Conic')

# run the tool
#arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)


C:\Users\and04671\Documents\Data_Files\AIANNH
C:\Users\and04671\Documents\Data_Files\AIANNH\tl_2018_us_aiannh_proj.shp
C:\Users\and04671\Documents\Data_Files\AITSN
C:\Users\and04671\Documents\Data_Files\AITSN\tl_2018_us_aitsn_proj.shp
C:\Users\and04671\Documents\Data_Files\ANRC
C:\Users\and04671\Documents\Data_Files\ANRC\tl_2018_02_anrc_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01001_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01003_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01005_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01007_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01009_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01011_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_01013_areawater_proj.shp
C:\Users\and04671\Documents\D

C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04001_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04003_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04005_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04007_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04009_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04011_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04012_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04013_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04015_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04017_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04019_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_04021_areawater_proj.shp
C:\Users\and0467

C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06021_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06023_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06025_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06027_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06029_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06031_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06033_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06035_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06037_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06039_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06041_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_06043_areawater_proj.shp
C:\Users\and0467

C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08103_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08105_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08107_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08109_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08111_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08113_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08115_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08117_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08119_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08121_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08123_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_08125_areawater_proj.shp
C:\Users\and0467

C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13019_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13021_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13023_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13025_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13027_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13029_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13031_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13033_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13035_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13037_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13039_areawater_proj.shp
C:\Users\and04671\Documents\Data_Files\AREAWATER\tl_2018_13043_areawater_proj.shp
C:\Users\and0467

...

C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55067_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55069_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55071_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55073_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55075_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55077_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55078_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55079_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55081_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55083_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_55085_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_

C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72053_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72054_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72055_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72057_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72059_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72061_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72063_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72065_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72067_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72069_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_2018_72071_linearwater_proj.shp
C:\Users\and04671\Documents\Data_Files\LINEARWATER\tl_

C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_49_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_50_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_51_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_53_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_54_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_55_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_56_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_60_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_66_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_69_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_72_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PLACE\tl_2018_78_place_proj.shp
C:\Users\and04671\Documents\Data_Files\PUMA
C:\Users\and04671\Documents\Data_Files\PUMA\tl_2018_01_puma10_proj.shp
C:\Users\and04671\Documents\Data_

C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_34_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_35_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_36_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_37_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_38_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_39_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_40_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_41_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_42_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_44_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_45_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_46_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_47_sldl_proj.shp
C:\Users\and04671\Documents\Data_Files\SLDL\tl_2018_48_sldl_proj.shp
C:\Users\and04671\Documents\Data_F

C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_42_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_44_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_45_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_46_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_47_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_48_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_49_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_50_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_51_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_53_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_54_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_55_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_56_tract_proj.shp
C:\Users\and04671\Documents\Data_Files\TRACT\tl_2018_60_tract_proj.shp
C:\Use

## Water merge

In [ ]:
import os
import sys
import zipfile
import string
import arcpy

#constants
working_directory = "C:/Users/and04671/Documents/Data_Files"
out_db = "C:/Users/and04671/Documents/Data_Files/TIGER2018_watermerge"
water = 'AREAWATER'

shp_dir = working_directory + "/" + water
#

arcpy.env.workspace = shp_dir
shps = arcpy.ListFeatureClasses("*")
out_fc = out_db + "tl_2017_us_" + water.lower()
#arcpy.Merge_management(shps,out_fc)


## Rename Statewide

In [ ]:

# This script renames the state-specific block group feature classes by
# postal abbreviations

import os
import sys
import zipfile
import string
import arcpy

working_directory = r"C:/Users/and04671/Documents/Data_Files"
geodb = working_directory + "TIGER2018_proj_watererase.gdb"
out_fc_stub = "_blck_grp_2018"

state_codes = {
 '53':'WA',
 '10':'DE',
 '11':'DC',
 '55':'WI',
 '54':'WV',
 '15':'HI',
 '12':'FL',
 '56':'WY',
 '72':'PR',
 '34':'NJ',
 '35':'NM',
 '48':'TX',
 '22':'LA',
 '37':'NC',
 '38':'ND',
 '31':'NE',
 '47':'TN',
 '36':'NY',
 '42':'PA',
 '02':'AK',
 '32':'NV',
 '33':'NH',
 '51':'VA',
 '08':'CO',
 '06':'CA',
 '01':'AL',
 '05':'AR',
 '50':'VT',
 '17':'IL',
 '13':'GA',
 '18':'IN',
 '19':'IA',
 '25':'MA',
 '04':'AZ',
 '16':'ID',
 '09':'CT',
 '23':'ME',
 '24':'MD',
 '40':'OK',
 '39':'OH',
 '49':'UT',
 '29':'MO',
 '27':'MN',
 '26':'MI',
 '44':'RI',
 '20':'KS',
 '30':'MT',
 '28':'MS',
 '45':'SC',
 '21':'KY',
 '41':'OR',
 '46':'SD',
 '72':'PR' 
}

arcpy.env.workspace = geodb
fcs = arcpy.ListFeatureClasses("bg_*")

for fc in fcs:
    fc_name = fc.split("_")
    fc_state = fc_name[1]
    state_abbr = state_codes.get(fc_state)
    out_fc = state_abbr + out_fc_stub
    arcpy.Rename_management(fc,out_fc)
    



## Export to SHP .zip

In [ ]:

# This script exports all fcs in a geodb to shapefiles and ZIPS them using the
# NHGIS naming convention

import os
import sys
import glob
import zipfile
import string
import arcpy

working_directory = r"C:/Users/and04671/Documents/Data_Files"
#geodb = wd + "TIGER2018_proj_watererase.gdb"
out_folder = working_directory + "/shps/"

#arcpy.env.workspace = geodb
#fcs = arcpy.ListFeatureClasses("*")

#for fc in fcs:
#    out_shp = fc + ".shp"
#    arcpy.FeatureClassToFeatureClass_conversion(fc,out_folder,out_shp)

os.chdir(out_folder)

shps = glob.glob(out_folder + "*.shp")

ziplist = []

for name in shps:
    file = os.path.basename(name)
    names = file[:-4]
    ziplist.append(names)

for f in ziplist:
    file_name = os.path.join(out_folder,f + ".zip")
    files = glob.glob(str(f)+".*")
    zips = zipfile.ZipFile(file_name,'w')
    for s in files:
        zips.write(s, compress_type = zipfile.ZIP_DEFLATED)
    zips.close()    



## Rename .zips

In [ ]:

# This script renames shapefiles to lower case 'us' and three character
# fips codes. 

import os
import sys
import glob
import zipfile
import string
#import arcpy

states_upper = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY", "PR"]

state_codes = {
    'WA': '530',
    'DE': '100',
    'DC': '110',
    'WI': '550',
    'WV': '540',
    'HI': '150',
    'FL': '120',
    'WY': '560',
    'PR': '720',
    'NJ': '340',
    'NM': '350',
    'TX': '480',
    'LA': '220',
    'NC': '370',
    'ND': '380',
    'NE': '310',
    'TN': '470',
    'NY': '360',
    'PA': '420',
    'AK': '020',
    'NV': '320',
    'NH': '330',
    'VA': '510',
    'CO': '080',
    'CA': '060',
    'AL': '010',
    'AR': '050',
    'VT': '500',
    'IL': '170',
    'GA': '130',
    'IN': '180',
    'IA': '190',
    'MA': '250',
    'AZ': '040',
    'ID': '160',
    'CT': '090',
    'ME': '230',
    'MD': '240',
    'OK': '400',
    'OH': '390',
    'UT': '490',
    'MO': '290',
    'MN': '270',
    'MI': '260',
    'RI': '440',
    'KS': '200',
    'MT': '300',
    'MS': '280',
    'SC': '450',
    'KY': '210',
    'OR': '410',
    'SD': '460',
    'PR': '720'
}

SHPwd = "C:/Users/and04671/Documents/Data_Files/shps"

os.chdir(wd)

zips = glob.glob(wd + "US*.zip")

ziplist = []

for name in zips:
    file = os.path.basename(name)
    ziplist.append(file)

for f in ziplist:
    new_zip = f.lower()
    os.rename(f,new_zip)

for state in states_upper:
    in_zip = glob.glob(wd + state + "*.zip")
    in_zip_string = in_zip[0]
    in_zip_file = os.path.basename(in_zip_string)

    out_zip_file = state_codes.get(state) + "_blck_grp_2017.zip"
    os.rename(in_zip_file, out_zip_file)
    
                 